In [1]:
# import firebase_admin
# from firebase_admin import credentials, firestore
# from flask import Flask, request, jsonify
# import joblib
# from sklearn.feature_extraction.text import TfidfVectorizer
# from firebase_admin import auth

# app = Flask(__name__)

# # Initialize Firebase Admin SDK and Firestore
# if not firebase_admin._apps:
#     cred = credentials.Certificate('firebase_credentials.json')  # Path to your Firebase credentials JSON
#     firebase_admin.initialize_app(cred)

# # Initialize Firestore
# db = firestore.client()

# # Load the trained SVM model and TF-IDF vectorizer
# svm_model = joblib.load('spam_classifier_model.pkl')
# vectorizer = joblib.load('tfidf_vectorizer.pkl')



# @app.route('/register', methods=['POST'])
# def register_user():
#     data = request.json
#     email = data['email']
#     password = data['password']

#     try:
#         # Create the user in Firebase Authentication
#         user = auth.create_user(email=email, password=password)

#         # Call function to add a document in Firestore
#         create_user_document(user.uid, email)

#         return jsonify({"status": "success", "message": "User registered successfully!"}), 200
#     except Exception as e:
#         return jsonify({"status": "error", "message": str(e)}), 400

# def create_user_document(uid, email):
#     # Ensure that the Firestore `users` collection contains a document for the new user
#     user_ref = db.collection('users').document(uid)
#     user_ref.set({
#         'email': email,
#         'created_at': firestore.SERVER_TIMESTAMP
#     })

# # Route to send an email
# @app.route('/send-email', methods=['POST'])
# def send_email():
#     data = request.json
#     sender = data['sender']
#     recipient = data['recipient']
#     subject = data['subject']
#     body = data['body']

#     # Classify the email
#     X = vectorizer.transform([body])
#     prediction = svm_model.predict(X)
#     is_spam = prediction[0] == 1  # 1 = Spam, 0 = Not Spam

#     # Email data to save
#     email_data = {
#         'sender': sender,
#         'recipient': recipient,
#         'subject': subject,
#         'body': body,
#         'received_at': firestore.SERVER_TIMESTAMP,
#         'isSpam': is_spam
#     }

#     # Add to global "emails" collection
#     db.collection('emails').add(email_data)

#     # Ensure sender and recipient documents exist in "users" collection
#     db.collection('users').document(sender).set({}, merge=True)
#     db.collection('users').document(recipient).set({}, merge=True)

#     # Add email to recipient's sub-collection
#     recipient_collection = 'spam' if is_spam else 'inbox'
#     db.collection('users').document(recipient).collection(recipient_collection).add(email_data)

#     # Add email to sender's 'sent' sub-collection
#     db.collection('users').document(sender).collection('sent').add(email_data)

#     return jsonify({"status": "success", "message": "Email sent successfully!"}), 200


# # Route to get sent emails for a specific user (sender)
# @app.route('/sent-emails/<user_id>', methods=['GET'])
# def get_sent_emails_for_user(user_id):
#     # Retrieve sent emails from the sender's 'sent' sub-collection
#     sent_emails_ref = db.collection('users').document(user_id).collection('sent')
#     sent_emails = [email.to_dict() for email in sent_emails_ref.stream()]

#     if not sent_emails:
#         return jsonify({"status": "no emails found"}), 404

#     return jsonify(sent_emails), 200


# # Route to get emails in the inbox for a specific user
# @app.route('/inbox/<user_id>', methods=['GET'])
# def get_inbox(user_id):
#     # Retrieve emails from the user's 'inbox' sub-collection
#     inbox_ref = db.collection('users').document(user_id).collection('inbox')
#     inbox_emails = [email.to_dict() for email in inbox_ref.stream()]

#     if not inbox_emails:
#         return jsonify({"status": "no emails found"}), 404

#     return jsonify(inbox_emails), 200


# # Route to get spam emails for a specific user
# @app.route('/spam/<user_id>', methods=['GET'])
# def get_spam(user_id):
#     # Retrieve spam emails from the user's 'spam' sub-collection
#     spam_ref = db.collection('users').document(user_id).collection('spam')
#     spam_emails = [email.to_dict() for email in spam_ref.stream()]

#     if not spam_emails:
#         return jsonify({"status": "no spam emails found"}), 404

#     return jsonify(spam_emails), 200


# if __name__ == '__main__':
#     app.run(debug=False)


In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, auth
from flask import Flask, request, jsonify
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": ["http://localhost:3002", "http://127.0.0.1:3002"]}})

# Allow CORS for specific origins

# # Initialize Flask App
# app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Initialize Firebase Admin SDK and Firestore
if not firebase_admin._apps:
    cred = credentials.Certificate('firebase_credentials.json')  # Path to your Firebase credentials JSON
    firebase_admin.initialize_app(cred)

db = firestore.client()

# Load the trained SVM model and TF-IDF vectorizer
svm_model = joblib.load('spam_classifier_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')


@app.route('/register', methods=['POST'])
def register_user():
    data = request.json
    email = data['email']
    password = data['password']

    try:
        # Create the user in Firebase Authentication
        user = auth.create_user(email=email, password=password)

        # Call function to add a document in Firestore
        create_user_document(user.uid, email)

        return jsonify({"status": "success", "message": "User registered successfully!"}), 200
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 400


def create_user_document(uid, email):
    # Ensure that the Firestore `users` collection contains a document for the new user
    user_ref = db.collection('users').document(uid)
    user_ref.set({
        'email': email,
        'created_at': firestore.SERVER_TIMESTAMP
    })


# Route to send an email
# @app.route('/send-email', methods=['POST'])
# def send_email():
#     try:
#         data = request.json
#         sender = data['sender']
#         recipient = data['recipient']
#         subject = data['subject']
#         body = data['body']

#         # Classify the email
#         X = vectorizer.transform([body])
#         X_dense = X.toarray()  # Convert sparse matrix to dense array
#         prediction = svm_model.predict(X_dense)
#         is_spam = bool(prediction[0] == 1)  # Explicitly convert to Python bool

#         # Email data to save
#         email_data = {
#             'sender': sender,
#             'recipient': recipient,
#             'subject': subject,
#             'body': body,
#             'received_at': firestore.SERVER_TIMESTAMP,
#             'isSpam': is_spam
#         }

#         # Add to global "emails" collection
#         db.collection('emails').add(email_data)

    #     # Ensure sender and recipient documents exist in "users" collection
    #     db.collection('users').document(sender).set({}, merge=True)
    #     db.collection('users').document(recipient).set({}, merge=True)

    #     # Add email to recipient's sub-collection
    #     recipient_collection = 'spam' if is_spam else 'inbox'
    #     db.collection('users').document(recipient).collection(recipient_collection).add(email_data)

    #     # Add email to sender's 'sent' sub-collection
    #     db.collection('users').document(sender).collection('sent').add(email_data)

    #     return jsonify({"status": "success", "message": "Email sent successfully!"}), 200
    # except Exception as e:
    #     return jsonify({"status": "error", "message": str(e)}), 500


@app.route('/send-email', methods=['POST'])
def send_email():
    try:
        data = request.json
        sender = data['sender']
        recipient = data['recipient']
        subject = data['subject']
        body = data['body']

        # Classify the email
        X = vectorizer.transform([body])
        X_dense = X.toarray()  # Convert sparse matrix to dense array
        prediction = svm_model.predict(X_dense)
        is_spam = bool(prediction[0] == 1)  # Explicitly convert to Python bool

        #print(f"Classification Result: {'Spam' if is_spam else 'Ham'}")  # Debugging log

        # Email data to save
        email_data = {
            'sender': sender,
            'recipient': recipient,
            'subject': subject,
            'body': body,
            'received_at': firestore.SERVER_TIMESTAMP,
            'isSpam': is_spam
        }

        # Add to global "emails" collection
        db.collection('emails').add(email_data)

        # Ensure sender and recipient documents exist in "users" collection
        db.collection('users').document(sender).set({}, merge=True)
        db.collection('users').document(recipient).set({}, merge=True)

        # Add email to recipient's sub-collection
        recipient_collection = 'spam' if is_spam else 'inbox'
        db.collection('users').document(recipient).collection(recipient_collection).add(email_data)

        # Add email to sender's 'sent' sub-collection
        db.collection('users').document(sender).collection('sent').add(email_data)

        return jsonify({"status": "success", "message": "Email sent successfully!"}), 200
    except Exception as e:
        print(f"Error: {str(e)}")  # Debugging log
        return jsonify({"status": "error", "message": str(e)}), 500




# Route to get sent emails for a specific user (sender)
@app.route('/sent-emails/<user_id>', methods=['GET'])
def get_sent_emails_for_user(user_id):
    try:
        sent_emails_ref = db.collection('users').document(user_id).collection('sent')
        sent_emails = [email.to_dict() for email in sent_emails_ref.stream()]

        if not sent_emails:
            return jsonify({"status": "no emails found"}), 404

        return jsonify(sent_emails), 200

    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500


# Route to get emails in the inbox for a specific user
@app.route('/inbox/<user_id>', methods=['GET'])
def get_inbox(user_id):
    try:
        inbox_ref = db.collection('users').document(user_id).collection('inbox')
        inbox_emails = [email.to_dict() for email in inbox_ref.stream()]

        if not inbox_emails:
            return jsonify({"status": "no emails found"}), 404

        return jsonify(inbox_emails), 200

    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500


# Route to get spam emails for a specific user
@app.route('/spam/<user_id>', methods=['GET'])
def get_spam(user_id):
    try:
        spam_ref = db.collection('users').document(user_id).collection('spam')
        spam_emails = [email.to_dict() for email in spam_ref.stream()]

        if not spam_emails:
            return jsonify({"status": "no spam emails found"}), 404

        return jsonify(spam_emails), 200

    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/classify', methods=['POST'])
def classify_email():
    data = request.json
    email_content = data.get("email_content", "")
    
    # Your spam classification logic here
    is_spam = svm_model.predict([email_content])  # Replace with your SVM logic
    
    # Return the classification result
    return jsonify({"isSpam": bool(is_spam)})

# try:
#     # Check Firebase initialization
#     if not firebase_admin._apps:
#         cred = credentials.Certificate('firebase_credentials.json')
#         firebase_admin.initialize_app(cred)
#         print("Firebase initialized successfully.")

#     # Load SVM and vectorizer models
#     svm_model = joblib.load('spam_classifier_model.pkl')
#     vectorizer = joblib.load('tfidf_vectorizer.pkl')
#     print("Models loaded successfully.")
# except Exception as e:
#     print(f"Error during initialization: {str(e)}")


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5002, debug=False)

In [7]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, Flask!"

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [2]:
%tb



SystemExit: 1